In [5]:
# Parameters

frequency = 'day'
resolution = '44i' #44i = 0.5; 22i = 0.25 lat/lon
bias_correction = 'raw' # raw, mbcn-Daymet, or mbcn-gridMET

simulations = ['CanESM2.CanRCM4', 'CanESM2.RCA4']

In [6]:
import intake
import pandas as pd

# need AWS access keys set up in environ to pull zarr files
zarr_urls = {
    'rcp45_prec':f's3://ncar-na-cordex/day/prec.rcp45.{frequency}.NAM-{resolution}.{bias_correction}.zarr',
    'rcp85_prec':f's3://ncar-na-cordex/day/prec.rcp85.{frequency}.NAM-{resolution}.{bias_correction}.zarr',
    'rcp45_temp':f's3://ncar-na-cordex/day/temp.rcp45.{frequency}.NAM-{resolution}.{bias_correction}.zarr',
    'rcp85_temp':f's3://ncar-na-cordex/day/temp.rcp85.{frequency}.NAM-{resolution}.{bias_correction}.zarr'
}

datasets = {}

for key, url in zarr_urls.items():

    zarr_cat = intake.open_zarr(url)
    zarr_source = zarr_cat()
    
    try:
        # Attempt to load the dataset
        dataset = zarr_source.to_dask()
        datasets[key] = dataset
        print(f"Dataset from {url} loaded successfully")

    except Exception as e:
        print(f"Error loading dataset from {url}: {e}")

Dataset from s3://ncar-na-cordex/day/prec.rcp45.day.NAM-44i.raw.zarr loaded successfully
Dataset from s3://ncar-na-cordex/day/prec.rcp85.day.NAM-44i.raw.zarr loaded successfully
Dataset from s3://ncar-na-cordex/day/temp.rcp45.day.NAM-44i.raw.zarr loaded successfully
Dataset from s3://ncar-na-cordex/day/temp.rcp85.day.NAM-44i.raw.zarr loaded successfully


In [7]:
# Get dataset, filter, aggregate with xarray
rcp45_prec_ds = (
    datasets.get('rcp45_prec')
        .sel(
            lat = slice(37,41), 
            lon = slice(-105,-102), 
            member_id = simulations, 
            bnds = 0)
        .drop('time_bnds')
        .mean(dim=['lat', 'lon'])
)

rcp85_prec_ds = (
    datasets.get('rcp85_prec')
        .sel(
            lat = slice(37,41), 
            lon = slice(-105,-102), 
            member_id = simulations, 
            bnds = 0)
        .drop('time_bnds')
        .mean(dim=['lat', 'lon'])
)

rcp45_temp_ds = (
    datasets.get('rcp45_temp')
        .sel(
            lat = slice(37,41), 
            lon = slice(-105,-102), 
            member_id = simulations, 
            bnds = 0)
        .drop('time_bnds')
        .mean(dim=['lat', 'lon'])
)

rcp85_temp_ds = (
    datasets.get('rcp85_temp')
        .sel(
            lat = slice(37,41), 
            lon = slice(-105,-102), 
            member_id = simulations,
              bnds = 0)
        .drop('time_bnds')
        .mean(dim=['lat', 'lon'])
)

ds_dict = {
    'rcp45_prec_ds':rcp45_prec_ds,
    'rcp45_temp_ds':rcp45_temp_ds,
    'rcp85_prec_ds':rcp85_prec_ds,
    'rcp85_temp_ds':rcp85_temp_ds
}

In [8]:
for name, ds in ds_dict.items():
    for id in ds.member_id:
        (ds.sel(member_id = id)
            .drop_vars('member_id')
            .to_dataframe()
            .reset_index()
            .to_parquet(
                f'data/scenarios/{name}_{frequency}_{resolution}_{bias_correction}_{id.values}.parquet'))